In [0]:
import os

# Import files from Google drive
from google.colab import drive
drive.mount('/content/drive/')
data_csv = '/content/drive/My Drive/data_complete.csv'
people = os.listdir('/content/drive/My Drive/people')
mask_rcnn_coco = '/content/drive/My Drive/mask_rcnn_coco.h5'

In [0]:
%tensorflow_version 1.x
import tensorflow

!pip install q keras==2.2.5

!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!pip show mask-rcnn

In [0]:
from os import listdir
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
import pandas as pd
from mrcnn.visualize import display_instances
from mrcnn.visualize import plot_precision_recall
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from numpy import expand_dims
from numpy import mean
from mrcnn.model import mold_image
from matplotlib.patches import Rectangle
from numpy import expand_dims

%cd ..

In [0]:
data_csv = pd.read_csv(data_csv) 
class_csv = data_csv.loc[: , "class"]
filenames_csv = data_csv.loc[: , "filename"]
xmin = data_csv.loc[: , "xmin"]
xmax = data_csv.loc[: , "xmax"]
ymin = data_csv.loc[: , "ymin"]
ymax = data_csv.loc[: , "ymax"]
width_csv = data_csv.loc[: , "width"]
height_csv = data_csv.loc[: , "height"]

In [0]:
class people(Dataset):
  def load_dataset(self, dataset_dir, is_train=True):
    self.add_class("dataset", 1, "person")
    self.add_class("dataset", 2, "head")
    self.add_class("dataset", 3, "hand")
    self.add_class("dataset", 4, "foot")
		# define data locations
    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annotations/'
    counter = 1
    for filename in listdir(images_dir):
			# skip all images after 150 if we are building the train set
      if is_train and counter >= 456:
        continue
			# skip all images before 150 if we are building the test/val set
      if not is_train and counter < 456:
        counter += 1
        continue
      counter += 1
      img_path = images_dir + filename
      ann_path = annotations_dir + filename[:-4] + '.xml'
			# add to dataset
      self.add_image('dataset', image_id=filename[:-4], path=img_path, annotation=ann_path)


	# extract bounding boxes 
  def extract_boxes(self, filename):
        boxes = list()
        clss_list = list()
        for i, item in enumerate(filenames_csv):
            if item[-15:-4] == filename[-15:]:
                coors = [xmin[i], ymin[i], xmax[i], ymax[i]]
                clss = class_csv[i]
                width = width_csv[i]
                height = height_csv[i]
                boxes.append(coors)
                clss_list.append(clss)
        return boxes, width, height, clss_list

	# load the masks for an image
  def load_mask(self, image_id):
		# get details of image
        info = self.image_info[image_id]
		# define box file location
        image_name = info['id']
		# load XML
        boxes, w, h, clss_list = self.extract_boxes(image_name)
		# create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(clss_list[i]))
        return masks, asarray(class_ids, dtype='int32')

	# load an image reference
  def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

# define a configuration for the model
class PeopleConfig(Config):
	# define the name of the configuration
	NAME = "people_cfg"
	# number of classes (background + people, head, hands, foot)
	NUM_CLASSES = 1 + 4
	# number of training steps per epoch
	STEPS_PER_EPOCH = 455


# train set
train_set = people()
train_set.load_dataset('drive/My Drive/people', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# test set
test_set = people()
test_set.load_dataset('drive/My Drive/people', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

In [0]:
for i in range(1,5): 
  image_id = i
  # load the image
  image = train_set.load_image(image_id)
  # load the masks and the class ids
  mask, class_ids = train_set.load_mask(image_id)
  # extract bounding boxes from the masks
  bbox = extract_bboxes(mask)
  # display image with masks and bounding boxes
  display_instances(image, bbox, mask, class_ids, train_set.class_names)

In [0]:
class PeopleConfig(Config):
    # define the name of the configuration
    NAME = "std_model"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 4
    # number of training steps per epoch
    STEPS_PER_EPOCH = 455
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }
  # prepare config
config = PeopleConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights(mask_rcnn_coco, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=10, layers='heads')

In [0]:
# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "people_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 4
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

def evaluate_model(dataset, model, cfg):
 APs = list()
 #precs = list()
 #recs = list()
 #overls = list()
 for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, prec, rec, overl = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
		#precs.append(prec)
		#recs.append(rec)
		#overls.append(overl)
	# calculate the mean AP across all images
 mAP = mean(APs)
 return mAP, prec, rec, overl

In [0]:
for i in range(5,10):
  # create config
  cfg = PredictionConfig()
  # define the model
  model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
  # load model weights
  model.load_weights('std_model1153151/mask_rcnn_std_model1155151_000' + str(i) + '.h5', by_name=True)
  # evaluate model on training dataset
  train_mAP, train_precs, train_recs, train_overls = evaluate_model(train_set, model, cfg)
  print("Train mAP: %.3f" % train_mAP)
  # evaluate model on test dataset
  test_mAP, test_precs, test_recs, test_overls = evaluate_model(test_set, model, cfg)
  print("Test mAP: %.3f" % test_mAP)
  print(" ")
plot_precision_recall(train_mAP, train_precs, train_recs)
plot_precision_recall(test_mAP, test_precs, test_recs)

In [0]:
classes = ['person', 'head', 'hand', 'foot']

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted_2(dataset, model, cfg, image_numb):
  # load the image and mask
  image = dataset.load_image(image_numb)
  mask, _ = dataset.load_mask(image_numb)
  # convert pixel values (e.g. center)
  scaled_image = mold_image(image, cfg)
  # convert image into one sample
  sample = expand_dims(scaled_image, 0)
  # make prediction
  yhat = model.detect(sample, verbose=0)[0]
  # define subplot
  #pyplot.subplot(1, 2, 1)
  # plot raw pixel data
  pyplot.imshow(image)
  pyplot.title('Actual')
  # plot masks
  for j in range(mask.shape[2]):
    pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
  pyplot.show()
  # get the context for drawing boxes
  #pyplot.subplot(1, 2, 2)
  # plot raw pixel data
  pyplot.imshow(image)
  pyplot.title('Predicted')
  ax = pyplot.gca()
  # plot each box
  for val, box in enumerate(yhat['rois']):
    # get coordinates
    y1, x1, y2, x2 = box
    # calculate width and height of the box
    width, height = x2 - x1, y2 - y1
    # create the shape
    rect = Rectangle((x1, y1), width, height, fill=False, color='blue')
    labelnumb = yhat["class_ids"][val]
    label = classes[labelnumb-1]
    ax.text(x1, y1 + 8, label,color='red', size=11, backgroundcolor="none")
    # draw the box
    ax.add_patch(rect)
  # show the figure
  pyplot.show()

In [0]:
import random
model_path = 'std_model/mask_rcnn_std_model_0007.h5'
model.load_weights(model_path, by_name=True)
counter = 0
i = 30
while counter < 30:
  #i = random.randrange(0,155)
  i = i + 1
  plot_actual_vs_predicted_2(test_set, model, cfg, i)
  counter += 1